### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
#%load_ext pycodestyle_magic

In [547]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=0, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_c = l_c.astype('float') / l_s
        r_c = r_c.astype('float') / r_s
        
        return (l_s*(np.dot(l_c, (1 - l_c).T))+
                r_s*(np.dot(r_c, (1 - r_c).T)))/(l_s + r_s)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        h_r = - np.dot(r_c / r_s, np.log(r_c / r_s).transpose())
        h_l = - np.dot(l_c / l_s, np.log(l_c / l_s).transpose())
        return (l_s * h_l + r_s * h_r) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        h_r = 1 - np.max(r_c / r_s)
        l_r = 1 - np.max(l_c / l_s)
        return (l_s * h_l + r_s * h_r) / (l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature) + 1)]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort(axis=0)
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    
    def __find_threshold(self, x, y):
        x_sort, y_sort=self.__sort_samples(x,y)
        #print(x_sort)
        ids_features = self.get_feature_ids(len(x[0]))
        best_value = 1
        best_feature = -1
        best_threshold = 0
        
        for feature in ids_features:
            mas  = x[:, feature]
            thresholds = [np.mean(mas), np.median(mas)]
            for threshold in thresholds:
                l_x, r_x, l_y, r_y = self.__div_samples(x, y, feature, threshold)
                value = self.G_function(np.bincount(l_y), l_y.size, np.bincount(r_y), r_y.size)
                if value < best_value:
                    best_value = value
                    best_feature = feature
                    best_threshold = threshold
        return best_feature, best_threshold

    def Is_leaf(self, y, depth):
        return (0 != self.max_depth < depth) or (np.unique(y).shape[0] == 1) or (y.size <= self.min_samples_split)
    
    def __fit_node(self, x, y, node_id, depth):
        
        if self.Is_leaf(y, depth):
            self.tree[node_id] = [self.__class__.LEAF_TYPE, np.argmax(np.bincount(y))]
        else:
            feature, threshold = self.__find_threshold(x, y)
            self.tree[node_id] = [self.__class__.NON_LEAF_TYPE, feature, threshold]
            l_x, r_x, l_y, r_y = self.__div_samples(x, y, feature, threshold)
            self.__fit_node(l_x, l_y, 2*node_id+1, depth+1)
            self.__fit_node(r_x, r_y, 2*node_id+2, depth+1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [548]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [554]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)
y_train

array([1, 0, 2, 0, 2, 1, 2, 0, 0, 2, 0, 0, 2, 1, 0, 0, 1, 1, 0, 1, 2, 1,
       0, 1, 0, 2, 2, 0, 0, 2, 1, 2, 1, 2, 0, 1, 1, 2, 1, 0, 1, 2, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 2, 1, 1, 2, 2, 1, 0, 1, 0, 1, 0, 0, 2,
       2, 1, 0, 1, 1, 1, 0, 0, 1, 2, 1, 1, 2, 2, 1, 1, 0, 0, 1, 2, 0, 1,
       2, 2, 2, 1, 0, 1, 0, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2,
       0, 2, 1, 2, 1, 0, 0, 1, 0, 0, 2, 1, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2,
       0, 2, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 2, 2,
       2, 1, 1, 2, 2, 1])

## Проверка скорости работы на wine

In [555]:
%time clf.fit(X_train, y_train)

Wall time: 2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [556]:
%time my_clf.fit(X_train, y_train)

Wall time: 23.9 ms


## Проверка качества работы на wine

In [557]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8363636363636364

In [558]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8885003885003885

## Подготовка данных Speed Dating Data 

## Проверка скорости работы на Speed Dating Data 

In [ ]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

In [ ]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

## Проверка качества работы на Speed Dating Data

In [ ]:
# тут должен быть код типа %time clf.fit(X_train, y_train)

In [ ]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)

## Задание 3

## Задание 4

## Задание 5